In [1]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = 'all' #默认为'last'
%config IPCompleter.greedy=True

In [5]:
import sys
import os
import pandas as pd
import numpy as np
# import pandas_profiling
import json
import math
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',1000)
import sklearn
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import train_test_split 
from collections import Counter
from pandas.api.types import is_dict_like
import itertools
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
sys.path.append('/Users/yantingting/PycharmProjects/modeling')
import plot_tools as pl
import metrics
import summary_statistics as ss
import feature_selection as fs
from data_io_utils import *
import data_processing as dp



In [6]:
file_path= '/Users/yantingting/Documents/海外风控/巴西/'
file_path_out = '/Users/yantingting/Documents/海外风控/巴西/Brazil_中间结果/'

# 巴西银行数据
不是所有的样本都用来进行分析，取消的订单不作为样本

In [ ]:
bank_data = pd.read_excel(file_path + '巴西字典.xlsx', sheet_name = 'PORTOCRED DATABASE',na_values='',dtype=({'ZIP CODE': str}))
var_mapping = pd.read_excel(file_path + 'bank_mapping.xlsx')                          
bank_data = bank_data.rename(columns = dict(zip(var_mapping['var_original'],var_mapping['var_name'] )))
drop_list= ['PROPOSTA CANCELADA',  'VALIDADE EXPIRADA', 'SIMULAÇÃO']  #剔除无效样本
bank_data = bank_data[~bank_data['var44'].isin(drop_list)]
reject_list = ['REPROVADO']
bank_data['flag_result'] = bank_data['var44'].apply(lambda x: 0 if x in reject_list else 1)
bank_data['var12'] = bank_data['var12'].apply(lambda x: x[x.find('@') + 1:x.find('.')] if not isinstance(x, float) else 0)
bank_data.to_csv(file_path_out + 'bank_data.csv')
#生成银行数据的所有loan
bank_data[['loan_id', 'DATE', 'flag_result']].to_csv(os.path.join(file_path_out, 'loan_bank.csv'), index = False)
print(bank_data.shape)
bank_data.head(2)

银行数据生成变量（未处理）

In [60]:
remain_col = ['loan_id', 'DATE','flag_result' , 'var17','var18','var9','var36']
var_bank_data = bank_data[remain_col].copy()
var_bank_data['age'] = bank_data.apply(lambda x: int(str(x['DATE'])[:4]) - pd.to_datetime(x['var1']).year, axis = 1)
var_bank_data['len_zipcode'] = bank_data['var6'].apply(lambda x: len(x) if not isinstance(x, float) else 0)
var_bank_data['society'] = bank_data['var37'].apply(lambda x: 0 if isinstance(x,float) else 1)
#one hot编码
ohe_index = ['var2','var3','var7','var10','var11','var12','var21','var26','var25','var22','var30','var31',
             'var32','var33','var34','var35','var37','var38',]
ohe_frame = pd.DataFrame()
for index in ohe_index:
    print(index)
    ohe_var = pd.get_dummies(bank_data[index])
    ohe_var.columns = [index +'-'+ str(i) for i in ohe_var.columns]
    ohe_frame = pd.concat([ohe_frame, ohe_var], axis=1)
var_bank_data = pd.concat([var_bank_data, ohe_frame], axis=1)

eda_result = ss.eda(var_bank_data)
drop_var = eda_result[eda_result['98%'] ==0]['var'].tolist()
drop_list = set(['loan_id', 'DATE', 'flag_result'] + drop_var)
var_dict_bank = dp.VarDict(var_bank_data.drop(drop_list, axis = 1), data_sorce='bank')
var_dict_bank.to_csv(os.path.join(file_path_out, 'var_dict_bank.csv'), index = False)
var_bank_data.drop(['DATE', 'flag_result'] + drop_var,axis =1).to_csv(os.path.join(file_path_out, 'var_bank_data.csv'), index = False)
var_bank_data.shape

var2
var3
var7
var10
var11
var12
var21
var26
var25
var22
var30
var31
var32
var33
var34
var35
var37
var38


(38676, 930)

In [62]:
var_dict_bank.head()

,数据源,指标英文,指标中文,数据类型
0,bank,var17,var17,float64
1,bank,var18,var18,float64
2,bank,var9,var9,float64
3,bank,var36,var36,float64
4,bank,age,age,float64
